In [1]:
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.signal import savgol_filter
import pandas as pd

In [3]:
def getDataDayMet(infiles, averaged=True):
    dat1 = pd.read_csv(infiles[0], sep='\s+', header=None, skiprows=6)
    dat2 = pd.read_csv(infiles[1], sep='\s+', header=None, skiprows=6)
    
    L = int(len(dat1[0]) % 365) + 1 
    
    DataOrig = dict()
    Data = dict()
    
    DataOrig['time'] = np.concatenate((dat1[0][:-L], dat2[0]))
    DataOrig['temp'] = np.concatenate((dat1[1][:-L], dat2[1]))
    
    DataOrig['time'] = np.array([1985 +  d/(86400*365) - 0.5 for d in DataOrig['time']])

    print len(DataOrig['temp'])%365

    for key in DataOrig.keys(): 
        if averaged == True:
            Data[key] = DataOrig[key].reshape((-1, 365)).mean(axis=1)
        else:
            Data[key] = DataOrig[key]
    
    return Data

In [4]:
DataOrig = dict()

In [5]:
def LoadData(infiles,averaged=True):
    global DataOrig
    dat1 = pd.read_csv(infiles[0], sep='\s+', header=None, skiprows=6)
    dat2 = pd.read_csv(infiles[1], sep='\s+', header=None, skiprows=6)
    
    L = int(len(dat1[0]) % 365) + 1
    L1 = int(len(dat2[0]) % 365)
    
    DataOrig = dict()
    Data = dict()
    L2 = 26*365
    DataOrig['time'] = np.concatenate((dat1[0][:L2], dat2[0]))
    DataOrig['temp'] = np.concatenate((dat1[1][:L2], dat2[1]))
        
    DataOrig['time'] = np.array([1985 + d/(86400*365) -0.5 for d in DataOrig['time']])

    for key in DataOrig.keys(): 
        if averaged == True:
            Data[key] = DataOrig[key].reshape((-1, 365)).mean(axis=1)
        else:
            Data[key] = DataOrig[key]
    return Data

In [6]:
MS_B_1R1 = dict()
MS_S30_1R1 = dict()
MS_S60_1R1 = dict()
MS_S90_1R1 = dict()

In [ ]:
def getData(DX, averaged=True):
    global MS_B_1R1
    global MS_S30_1R1
    global MS_S60_1R1
    global MS_S90_1R1
    
    infile1 = ['/Users/ajc/mytests/simulations/hillslope/2019/DaymetS2_R1/%s.dat'%DX]

    BaseS2_00 = getDataDayMet(infile1,averaged)

    infile1 = ['/Users/ajc/mytests/simulations/hillslope/2018/DaymetS2_R1/%s.dat'%DX,
               '/Users/ajc/mytests/simulations/hillslope/2019/FEB1/Feb1_S2_R3/%s.dat'%DX]

    Feb1_S2_00 = LoadData(infile1,averaged)


    infile1 = ['/Users/ajc/mytests/simulations/hillslope/2018//DaymetS2_R1/%s.dat'%DX, 
               '/Users/ajc/mytests/simulations/hillslope/2019/May10/May10_S2_R3/%s.dat'%DX]

    May10_S2_00 = LoadData(infile1,averaged)

    infile1 = ['/Users/ajc/mytests/simulations/hillslope/2018/DaymetS2_R1/%s.dat'%DX, 
               '/Users/ajc/mytests/simulations/hillslope/2019/SEPT27/Sept27_S2_R3/%s.dat'%DX]
    Sept27_S2_00 = LoadData(infile1,averaged)

    infile1 = ['/Users/ajc/mytests/simulations/hillslope/2018/DaymetS2_R1/%s.dat'%DX, 
               '/Users/ajc/mytests/simulations/hillslope/2019/NOV15/Nov15_S2_R3/%s.dat'%DX]
    Nov15_S2_00 = LoadData(infile1,averaged)

    infile1 = ['/Users/ajc/mytests/simulations/hillslope/2018/DaymetS2_R1/%s.dat'%DX, 
               '/Users/ajc/mytests/simulations/hillslope/2019/SEPT27NOV15/Sept27Nov15_S2_R3/%s.dat'%DX]
    Sept27Nov15_S2_R3_00 = LoadData(infile1,averaged)

In [ ]:
DX='temperature-dx19'
#DX='temperature-40cm'
DX='temperature-riparian-40cm'
DX='temperature-OM'
#DX='temperature-hillslope-40cm'
#DX='temperature-hillslope'
#DX='temperature-riparian-5m'
#DX='temperature-5m'
#DX='surface_flux-day'
#DX= 'surface-pond_size'
#DX='saturation_ice-40cm'


In [ ]:
def getpolyfit():
    infile1 = h5py.File('/Users/ajc/mytests/simulations/Data/barrow1985-2015/barrow2015-2016-snowevents-smooth.h5','r')
    infile2 = h5py.File('/Users/ajc/mytests/simulations/Data/barrow1985-2015/barrow1985-2015-trend-smooth.h5','r')
    infile3 = h5py.File('Barrow-2010_2016-updated.h5','r')

    Forcing = dict()
    for key in infile2.keys():
        d = np.array(infile2[key])
        Forcing[key] = d.reshape((-1,365)).mean(axis=1)

    Forcing3 = dict()
    for key in infile3.keys():
        d = np.array(infile3[key][:-323])
        Forcing3[key] = d.reshape((-1,365)).mean(axis=1)

    time = [t/(86400*365) for t in Forcing['time [s]']]
    #print np.mean(Forcing['air temperature [K]'][-3:]), Forcing['air temperature [K]']

    temp_fit1 = np.polyfit(time, Forcing['air temperature [K]'],1)
    temp_fit = np.poly1d(temp_fit1)
    time1 = np.insert(time, len(time), time[-1]+1)

    snow_fit1 = np.polyfit(Barrow['time'], Barrow['precip'],1)
    snow_fit = np.poly1d(snow_fit1)
    #plt.show()

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
markers=['None', '.', ',', '+', 'o', '*']
lines = ['-', '--', '-.']
def Plot():
    fig, ax = plt.subplots(1,1, figsize=(5,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    ax.plot(Feb1_S2_00['time'], Feb1_S2_00['temp']) #---
    plt.tight_layout()
    #axs = axs.ravel()

    #plt.subplot(111)
    ax.plot(Sept27_S2_00['time'], Sept27_S2_00['temp'], color ='c', marker=markers[0], linestyle=lines[0], label='Early fall')
    ax.plot(Nov15_S2_00['time'], Nov15_S2_00['temp'], color ='m', marker=markers[0],  linestyle=lines[0], label='Late fall')
    ax.plot(Feb1_S2_00['time'], Feb1_S2_00['temp'], color ='k', marker=markers[0], linestyle=lines[0], label='Early winter')
    ax.plot(May10_S2_00['time'], May10_S2_00['temp'], color ='g', marker=markers[0], linestyle=lines[0], label='Late winter')
    ax.plot(Sept27Nov15_S2_R3_00['time'], Sept27Nov15_S2_R3_00['temp'], color ='b', marker=markers[0],  linestyle=lines[0],label='Early, late fall')
    ax.plot(BaseS2_00['time'], BaseS2_00['temp'], color ='r', marker=markers[0],  linestyle=lines[0],label='Base')
    ax.axvspan(1985+ 26.,1985+30.2, facecolor='0.2', alpha=0.3)
    plt.xlabel('Year',fontsize=12, fontweight='normal')
    plt.ylabel('Annual average temperature [K]',fontsize=12, fontweight='normal')

    #plt.ylim([262.,270])
    #ax.set_xlim([-0.1,30.1])
    print np.linspace(1985,2015,31)
    #ax.set_xticklabels(np.linspace(1985,2015,31))
    plt.legend()
    plt.savefig('%s.pdf'%DX, bbox_inches='tight', dpi=10)
    plt.show()

In [ ]:
getData(DX, False)


In [ ]:
Plot()

In [ ]:
# PLOT SURFACE DISCHARGE
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
fig, ax = plt.subplots(1,1, figsize=(5,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)
ax.plot(Feb1_S2_00['time'], Feb1_S2_00['temp']) #---
plt.tight_layout()
#axs = axs.ravel()

#plt.subplot(111)
ax.plot(Feb1_S2_00['time'], Feb1_S2_00['temp'], 'k.-',label='Mid-winter')
ax.plot(May10_S2_00['time'], May10_S2_00['temp'], 'g.-', label='Late-winter')
ax.plot(Sept27_S2_00['time'], Sept27_S2_00['temp'], 'c.-', label='Early-fall')
ax.plot(Nov15_S2_00['time'], Nov15_S2_00['temp'], 'm.-', label='Late-fall')
ax.plot(Sept27Nov15_S2_R3_00['time'], Sept27Nov15_S2_R3_00['temp'], 'b.-', label='Early, late-fall')
ax.plot(BaseS2_00['time'], BaseS2_00['temp'], 'r-*', label='Base')
ax.axvspan(26.5,31.5, facecolor='0.2', alpha=0.3)
plt.xlabel('Year',fontsize=12, fontweight='normal')
plt.ylabel('Annual average temperature [K]',fontsize=12, fontweight='normal')


#plt.ylim([262.,270])
#plt.xlim([0.,31])
plt.legend()
plt.savefig('%s.pdf'%DX, bbox_inches='tight', dpi=10)


plt.show()

In [ ]:
BaseS2_00_Pc = dict()
Feb1_S2_00_Pc = dict()
May10_S2_00_Pc = dict()
Sept27_S2_00_Pc = dict()
Nov15_S2_00_Pc = dict()
Sept27Nov15_S2_00_Pc = dict()

In [ ]:
yr_n = 25
BaseS2_00_Pc['temp'] = [ (d - 0*BaseS2_00['temp'][yr_n]) for d in  BaseS2_00['temp'][yr_n:]]
Feb1_S2_00_Pc['temp'] = [ (d - 0*BaseS2_00['temp'][yr_n]) for d in  Feb1_S2_00['temp'][yr_n:]]
May10_S2_00_Pc['temp'] = [ (d - 0*May10_S2_00['temp'][yr_n]) for d in  May10_S2_00['temp'][yr_n:]]
Sept27_S2_00_Pc['temp'] = [ (d - 0*Sept27_S2_00['temp'][yr_n]) for d in  Sept27_S2_00['temp'][yr_n:]]
Nov15_S2_00_Pc['temp'] = [ (d - 0*Nov15_S2_00['temp'][yr_n]) for d in  Nov15_S2_00['temp'][yr_n:]]
Sept27Nov15_S2_00_Pc['temp'] = [ (d - 0*Sept27Nov15_S2_R3_00['temp'][yr_n]) for d in  Sept27Nov15_S2_R3_00['temp'][yr_n:]]

fig, axs = plt.subplots(1,2)
fig.tight_layout()


axs[0].plot(BaseS2_00['time'][yr_n:], BaseS2_00_Pc['temp'], 'r-*', label='base')
axs[0].plot(Feb1_S2_00['time'][yr_n:], Feb1_S2_00_Pc['temp'], 'k.-', label='Feb')
axs[0].plot(May10_S2_00['time'][yr_n:], May10_S2_00_Pc['temp'], 'g.-', label='May')
axs[0].plot(Sept27_S2_00['time'][yr_n:], Sept27_S2_00_Pc['temp'], 'c.-', label='Sept')
axs[0].plot(Nov15_S2_00['time'][yr_n:], Nov15_S2_00_Pc['temp'], 'm.-', label='Now')
axs[0].plot(Sept27Nov15_S2_R3_00['time'][yr_n:], Sept27Nov15_S2_00_Pc['temp'], 'y.-', label='Now')
plt.legend()
plt.show()

In [ ]:
#Mineral Soil Thermal conductivities
sat = [0.3, 0.7, 1.0]
TC_u = [0.3, 0.5, 1.0]
TC_f = [0.3, 0.65, 1.3]
pfit = np.polyfit(sat, TC_u,2)

p = np.poly1d(pfit)

plt.plot(sat, p(sat),'r')
plt.plot(sat, TC_u,'g*')
plt.show()

# F = 1.667 x^2 - 1.167 x + 0.5
print 'p: ',p

#-----------------------------------
pfit = np.polyfit(sat, TC_f,2)

p = np.poly1d(pfit)

plt.plot(sat, p(sat),'r')
plt.plot(sat, TC_f,'g*')
plt.show()

# F = 1.845 x^2 - 0.9702 x + 0.425 
print 'p: ',p




In [ ]:
#Peat Thermal conductivities
vwc = [4./100., 65/100, 70/100.]
por = 0.87
s = 
sat = [0.3, 0.7, 1.0]
TC_u = [0.1, 0.5, 0.6]
TC_f = [0.1, 0.65, ]
pfit = np.polyfit(sat, TC_u,2)

p = np.poly1d(pfit)

plt.plot(sat, p(sat),'r')
plt.plot(sat, TC_u,'g*')
plt.show()

# F = 1.667 x^2 - 1.167 x + 0.5
print 'p: ',p

#-----------------------------------
pfit = np.polyfit(sat, TC_f,2)

p = np.poly1d(pfit)

plt.plot(sat, p(sat),'r')
plt.plot(sat, TC_f,'g*')
plt.show()

# F = 1.845 x^2 - 0.9702 x + 0.425 
print 'p: ',p





In [ ]:
"""
#---
axins = zoomed_inset_axes(ax, 1.4, loc='upper center')
axins = zoomed_inset_axes(ax,1.4, loc='upper center')
axins.plot(Feb1_S2_00['time'], Feb1_S2_00['temp'])
x1, x2, y1, y2 = 25, 32, 265.15, 267.6 # specify the limits
axins.set_xlim(x1, x2) # apply the x-limits
axins.set_ylim(y1, y2) # apply the y-limits
plt.yticks(visible=False)
plt.xticks(visible=False)
#axins.xaxis.set_visible('False')
#axins.yaxis.set_visible('False')
mark_inset(ax, axins, loc1=2, loc2=4, fc="none", ec="0.5")
"""